In [ ]:
import flickrapi
from PIL import Image
import os
import urllib.request
import urllib


api_key = 'ee6ae6e01cb3ab4ac1a21a584c38cdf0'
secret = '9648a599079887ec'

# Поиск по слову
keyword = 'cats'

# Количество изображений для поиска (количество аккаунтов)
accounts_num = 300

# Количество изображений для скачивания на 1 аккаунт
images_num = 100

# Куда сохранять фотографии из альбомов
path_to_save = '/Users/user/Desktop/aylifind/flickr/'

flickr=flickrapi.FlickrAPI(api_key, secret, cache=True)
photos = flickr.walk(text=keyword,
                     tag_mode='all',
                     tags=keyword,
                     extras='url_c',
                     per_page=100,
                     sort='relevance')


def get_init_photos_urls(photos):
    # Получаем изображение из поиска по запросу keyword
    
    urls = []
    for i, photo in enumerate(photos):
        url = photo.get('url_c')
        if url == None:
            continue
        urls.append(url)
        if len(urls) > accounts_num:
            break
    return urls


def get_photo_id(url):
    photo_id = url.split('/')[-1].split('_')[0]
    return photo_id


def get_albums_ids_with_similar_photos(photo_id):
    # На изображение, найденное поиском, получаем ID альбома, в котором оно содержится
    
    albums_ids = []
    im = flickr.photos.getAllContexts(api_key=api_key, photo_id=photo_id, format='parsed-json')
    albums_num = len(im['set'])
    for album_num in range(albums_num):
        albums_ids.append(im['set'][album_num]['id'])
    return albums_ids


def get_user_id(photo_id):
    # Получаем информацию о пользователе, загрузившем данное изображение
    
    user_request = flickr.photos.getInfo(api_key=api_key, photo_id=photo_id, format='parsed-json')
    user_id = user_request['photo']['owner']['nsid']
    return user_id


def request_album_with_similar_photos(albums_ids, url_num):
    
    photos_urls = []
    curr_account_num = 0
    curr_image_num = 0
    # Берем каждый из альбомов, в котором находятся похожие к исходному изображения...
    for album_id in albums_ids:
        curr_account_num += 1
        os.mkdir(path_to_save + str(url_num) + '_' + str(curr_account_num))
        r = flickr.photosets.getPhotos(api_key=api_key, photoset_id=album_id, user_id=user_id, format='parsed-json')
        num_of_photos_in_album = len(r['photoset']['photo'])
        
        # ...Выводим каждое фото из альбома
        for photo_num in range(num_of_photos_in_album):
            if curr_image_num < images_num: # скачиваем только первые (images_num) фотографий
                curr_image_num += 1
                photo_id = r['photoset']['photo'][photo_num]['id']
                photo_sizes = flickr.photos.getSizes(api_key=api_key, photo_id=photo_id, format='parsed-json')
                photo_url = photo_sizes['sizes']['size'][-1]['source']
                photos_urls.append(photo_url)
                photo_file = urllib.request.urlretrieve(photo_url, path_to_save + str(url_num) + '_' + str(curr_account_num) + f'/{curr_image_num}.jpg')
    return photos_urls
                

url_num = 0
urls = get_init_photos_urls(photos)
for url in urls:
    url_num += 1
    photo_id = get_photo_id(url)
    albums_ids = get_albums_ids_with_similar_photos(photo_id)
    user_id = get_user_id(photo_id)
    albums_with_sim_photos = request_album_with_similar_photos(albums_ids, url_num)
    
    
    